# Question Answering with BERT
In this exercise, we will use a pretrained BERT model, finetuned on question answering, to identify the answer to a question in a paragraph.

In [4]:
%pip install -q torch
%pip install -q transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch

## Knowledge distillation

[Knowledge distillation](https://en.wikipedia.org/wiki/Knowledge_distillation) is a practice in deep learning to train a much smaller *student* model on the outputs of a large *teacher* model. In that way, one can reduce model parameters a lot while performance decreases only by a little.

We will use a distilled model: `distilbert-base-uncased-distilled-squad`. From the 5 parts of the name, we know that:
- it is a distilled version of the BERT model
- it used BERT-base as a teacher model
- it is uncased, so converts all inputs into lowercase
- it was distilled again during finetuning (normally it's only done once from the pretrained model)
- the model was finetuned on squad (v1.1), a question answering dataset

We will now download and initialize the model. Read the [model documentation](https://huggingface.co/distilbert-base-uncased-distilled-squad) on HuggingFace's model hub to see how it is used.

In [6]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-distilled-squad')
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')
model.eval()
pass  # Suppress output

## Extractive Question Answering

For extractive question answering, we are given a paragraph and a question, and have to find the answer to the question in the paragraph. Look at the paragraphs (starts of Wikipedia articles about New York City, the capybara, and american football) and questions that we will use for this exercise:

In [7]:
paragraphs = [
    "New York, often called New York City (NYC), is the most populous city in the United States. With a 2020 population of 8,804,190 distributed over 300.46 square miles (778.2 km2), New York City is also the most densely populated major city in the United States. Located at the southern tip of New York State, the city is based in the Eastern Time Zone and constitutes the geographical and demographic center of both the Northeast megalopolis and the New York metropolitan area, the largest metropolitan area in the world by urban landmass. With over 20.1 million people in its metropolitan statistical area and 23.5 million in its combined statistical area as of 2020, New York is one of the world's most populous megacities. New York City is a global cultural, financial, and media center with a significant influence on commerce, health care and life sciences, entertainment, research, technology, education, politics, tourism, dining, art, fashion, and sports. New York is the most photographed city in the world. Home to the headquarters of the United Nations, New York is an important center for international diplomacy, an established safe haven for global investors, and is sometimes described as the capital of the world.",
    "The capybara or greater capybara (Hydrochoerus hydrochaeris) is a giant cavy rodent native to South America. It is the largest living rodent and a member of the genus Hydrochoerus. The only other extant member is the lesser capybara (Hydrochoerus isthmius). Its close relatives include guinea pigs and rock cavies, and it is more distantly related to the agouti, the chinchilla, and the nutria. The capybara inhabits savannas and dense forests and lives near bodies of water. It is a highly social species and can be found in groups as large as 100 individuals, but usually lives in groups of 10–20 individuals. The capybara is hunted for its meat and hide and also for grease from its thick fatty skin. It is not considered a threatened species.",
    "American football (referred to simply as football in the United States and Canada), also known as gridiron, is a team sport played by two teams of eleven players on a rectangular field with goalposts at each end. The offense, the team with possession of the oval-shaped football, attempts to advance down the field by running with the ball or passing it, while the defense, the team without possession of the ball, aims to stop the offense's advance and to take control of the ball for themselves. The offense must advance at least ten yards in four downs or plays; if they fail, they turn over the football to the defense, but if they succeed, they are given a new set of four downs to continue the drive. Points are scored primarily by advancing the ball into the opposing team's end zone for a touchdown or kicking the ball through the opponent's goalposts for a field goal. The team with the most points at the end of a game wins.",
]
questions = [
    [
        "What is New York's population?",
        "How many people live in New York's metropolitan area?",
        "To what industries is New York a center?",
        "What is New York known for?",
        "What is New York also called?",
        "What is New York also sometimes called?",
        "What is New York also sometimes described?",
    ],
    [
        "What is the scientific name of the capybara?",
        "What family of animals does the capybara belong to?",
        "What are close relatives of the capybara?",
        "What is the size of groups that the capybara lives in?",
        "What is the capybara hunted for?",
    ],
    [
        "Under what name is American football also known?",
        "How many players are in a team?",
        "By what means can the offense advance?",
        "How many yards must a team advance in four downs?",
        "How does a team score points?",
        "How does a team win?",
    ],
]

## Answering Questions

For each of the paragraphs above, we will now answer the associated questions. Take a look at the [model documentation](https://huggingface.co/distilbert-base-uncased-distilled-squad). First, use HuggingFace's pipeline abstraction. Look at the outputs of your model and describe the information it returns.

In [11]:
from transformers import pipeline

question_answer = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad", device='mps')

for i, paragraph in enumerate(paragraphs):
    for question in questions[i]:
        result = question_answer(question=question, context=paragraph)
        print(f"Question: {question}")
        print(
            f"Answer: '{result['answer']}', "
            f"Score: {round(result['score'], 4)}, "
            f"Start: {result['start']}, "
            f"End: {result['end']}\n"
        )
    print("-" * 40)

Question: What is New York's population?
Answer: '8,804,190', Score: 0.9858, Start: 118, End: 127

Question: How many people live in New York's metropolitan area?
Answer: '20.1 million', Score: 0.5023, Start: 548, End: 560

Question: To what industries is New York a center?
Answer: 'financial, and media', Score: 0.0384, Start: 760, End: 780

Question: What is New York known for?
Answer: 'New York City (NYC)', Score: 0.1166, Start: 23, End: 42

Question: What is New York also called?
Answer: 'New York City', Score: 0.3205, Start: 23, End: 36

Question: What is New York also sometimes called?
Answer: 'New York City (NYC', Score: 0.3319, Start: 23, End: 41

Question: What is New York also sometimes described?
Answer: 'New York City (NYC', Score: 0.3236, Start: 23, End: 41

----------------------------------------
Question: What is the scientific name of the capybara?
Answer: 'Hydrochoerus hydrochaeris', Score: 0.3257, Start: 34, End: 59

Question: What family of animals does the capybara 

Now we will do the same by directly using the tokenizer and the model we loaded at the start. The model documentation also has an example to get you started (they use the wrong class for the model [as of October 2022]). Look at the outputs of the model. Perform the necessary steps to give an answer like the pipeline above. Do the answers match?

In [31]:
import torch.nn.functional as F

for i, paragraph in enumerate(paragraphs):
    print(paragraph)
    for question in questions[i]:    
        inputs = tokenizer(question, paragraph, return_tensors="pt")
        print(inputs.keys())
        with torch.no_grad():
            outputs = model(**inputs)
        
        start_probs = F.softmax(outputs.start_logits[0], dim=-1)
        start_prob = start_probs.max().item()
        
        end_probs = F.softmax(outputs.end_logits[0], dim=-1)
        end_prob = start_probs.max().item()
        
        answer_start_index = torch.argmax(outputs.start_logits)
        answer_end_index = torch.argmax(outputs.end_logits)
        
        predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
        answer = tokenizer.decode(predict_answer_tokens)
        
        print(f"Question: {question}")
        print(f"Answer: {answer}\n")
        print(f"Start: {answer_start_index}")
        print(f"Start prob: {start_prob:.2%}")
        print(f"End: {answer_end_index}")
        print(f"End prob: {end_prob:.2%}")

New York, often called New York City (NYC), is the most populous city in the United States. With a 2020 population of 8,804,190 distributed over 300.46 square miles (778.2 km2), New York City is also the most densely populated major city in the United States. Located at the southern tip of New York State, the city is based in the Eastern Time Zone and constitutes the geographical and demographic center of both the Northeast megalopolis and the New York metropolitan area, the largest metropolitan area in the world by urban landmass. With over 20.1 million people in its metropolitan statistical area and 23.5 million in its combined statistical area as of 2020, New York is one of the world's most populous megacities. New York City is a global cultural, financial, and media center with a significant influence on commerce, health care and life sciences, entertainment, research, technology, education, politics, tourism, dining, art, fashion, and sports. New York is the most photographed city

Some of the answers do not match, which tells us that pipeline's implementation does something different 
from what we did. Two things to note:
1. A different tokenization procedure seems to have been used, as the answer to the first question appears
without spaces in the pipeline result, and with spaces for the tokenizer + model output.
2. The start/end indices in the pipeline model are the character start/end indices, whereas the indices we
computed in the tokenizer + model code are the indices of the input tokens.

## Evaluation

Evaluation in extractive QA is done by comparing the token-level overlap between the reference answer (sometimes also called *ground truth* or *gold answer*) and the model's answer. We first compute the precision $P$ ("how many tokens in the model's answer also appear in the reference?") and the recall $R$ ("how many tokens in the reference also appear in the model's answer?"). Their harmonic mean is also called F1 score, which is our evaluation metric.

$$
\begin{align}
\text{P} &= \frac{\text{number of tokens in both answers}}{\text{number of tokens in the model's answer}} \\
\text{R} &= \frac{\text{number of tokens in both answers}}{\text{number of tokens in the reference answer}} \\
\text{F1} &= 2 \frac{PR}{P + R} \\
\end{align}
$$

**Task:** Define your own solution to three of the questions above, then compute the word-level F1 score for one of the model's answers for each of them. The final result is the average over all questions.

**Answer:**

1. Question: To what industries is New York a center?
    Gold answer: cultural, financial, and media
    Model's answer: financial, and media

Precision = 3/3 = 100%
Recall = 3/4 = 75%


In [38]:
precision_q1 = 3/3
recall_q1 = 3/4
f1 = 2 * (precision_q1 * recall_q1) / (precision_q1 + recall_q1)
print(f"{f1:.2%}")

85.71%


2. Question: What is the size of groups that the capybara lives in?
    Gold answer: 10-20
    Model's answer: 10-20 individuals

In [39]:
precision_q2 = 1/2
recall_q2 = 1/1
f1 = 2 * (precision_q2 * recall_q2) / (precision_q2 + recall_q2)
print(f"{f1:.2%}")

66.67%


2. Question: How does a team score points?
    Gold answer: touchdown or field goal
    Model's answer: advancing the ball into the opposing team's end zone for a touchdown

In [40]:
precision_q3 = 1/12
recall_q3 = 1/4
f1 = 2 * (precision_q3 * recall_q3) / (precision_q3 + recall_q3)
print(f"{f1:.2%}")

12.50%
